In [1]:
from google.colab import files
uploaded = files.upload()
import pandas as pd # Import the pandas library
df = pd.read_csv('TMSMM_class6_Amazon_Reviews_small.csv')

Saving TMSMM_class6_Amazon_Reviews_small.csv to TMSMM_class6_Amazon_Reviews_small (1).csv


In [3]:
import pandas as pd
import numpy as np

# load the data
reviews_datasets = pd.read_csv('TMSMM_class6_Amazon_Reviews_small.csv')
reviews_datasets = reviews_datasets.head(450)
reviews_datasets.dropna()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text;;
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...
16,17,B001GVISJM,A3KLWF6WQ5BNYO,Erica Neathery,0,0,2,1348099200,poor taste,I love eating them and they are good for watch...
...,...,...,...,...,...,...,...,...,...,...
414,420,B004X8TK8I,A3GIMXLOEPFJBV,Samuel Spade,0,0,4,1342396800,These are Famous for a reason.,These arrived on time and packaged in green pl...
433,439,B000G6RYNE,A926YP1BSD72Y,Shopper11,0,0,5,1321747200,Best gluten-free dairy-free chips,These are my favorite gluten-free dairy-free f...
437,443,B000G6RYNE,A391CLP4UMXJQG,Dane King,0,0,4,1300924800,Very good.,Salt and vinegar chips are definitely my favor...
442,448,B000G6RYNE,ACFPHOYVB71V4,karob,0,0,5,1292630400,BEST BUY in BBQ Chips,Tried many varieties of BBQ chips and hands do...


In [4]:
# inspect some data
reviews_datasets['Text;;'][433]

"These are my favorite gluten-free dairy-free flavored chips.  Many other salt and vinegar chips contain dairy so it's nice to find ones that I can eat.  The vinegar flavor is really intense and good.;;"

In [5]:
# create vocabulary of all the words in our data
# remove stopwords
# only include those words that appear in less than 80% of the document and appear in at least 2 documents
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(reviews_datasets['Text;;'].values.astype('U'))


In [6]:
# 450 documents, vocabluary of 216 words
doc_term_matrix


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 792 stored elements and shape (450, 216)>

In [7]:
# use LDA to create topics along with the probability distribution for each word in our vocabulary for each topic
from sklearn.decomposition import LatentDirichletAllocation

# parameter n_components specifies the number of categories, or topics, that we want our text to be divided into
# seed to get similar results
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)


LatentDirichletAllocation(n_components=5, random_state=42)

In [15]:
# randomly fetch words from our vocabulary
# it randomly fetches 10 words from our vocabulary
import random

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names_out())) # Use get_feature_names_out() instead of get_feature_names()
    print(count_vect.get_feature_names_out()[random_id]) # Use get_feature_names_out() instead of get_feature_names()

little
wonderful
add
added
stick
live
type
regular
ham
eat


In [16]:
# find 10 words with the highest probability for the first topic
first_topic = LDA.components_[0]


In [17]:
top_topic_words = first_topic.argsort()[-10:]



In [19]:
for i in top_topic_words:
    print(count_vect.get_feature_names_out()[i]) # Use get_feature_names_out() instead of get_feature_names()

good
salt
really
eat
don
like
just
sugar
great
br


In [21]:
# 10 words with highest probabilities for all the five topics
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names_out()[i] for i in topic.argsort()[-10:]]) # Use get_feature_names_out() instead of get_feature_names()
    print('\n')

Top 10 words for topic #0:
['good', 'salt', 'really', 'eat', 'don', 'like', 'just', 'sugar', 'great', 'br']


Top 10 words for topic #1:
['br', 'hard', 'filling', 'good', 'really', 'flavors', 'like', 'sweet', 'love', 'flavor']


Top 10 words for topic #2:
['definitely', 'br', 'cat', 'price', 'product', 'dry', 'balance', 'natural', 'dog', 'food']


Top 10 words for topic #3:
['flavor', 'coffee', 'got', 'tea', 'drink', 've', 'like', 'taste', 'product', 'good']


Top 10 words for topic #4:
['water', 'really', 'easy', 'used', 'product', 'great', 'like', 'love', 'taste', 'just']




In [22]:
# add a column to the original data frame that will store the topic for the text
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(450, 5)

In [23]:
# add a new column for topic in the data frame and assigns the topic value to each row in the column
reviews_datasets['Topic'] = topic_values.argmax(axis=1)